In [86]:
import re

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.multioutput import MultiOutputRegressor

In [87]:
df = pd.read_csv("data60.csv", low_memory=False, parse_dates=['ut_ms'])
df = df.sample(frac=0.001)

In [88]:
df = df.replace(to_replace='?', value=0)
df = df.apply(pd.to_numeric)
df = df.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))

In [89]:
df = df.fillna(df.mean())

In [5]:
# df['date'] = pd.to_datetime(df['ut_ms'], unit='ms')
# df.drop('ut_ms', axis=1, inplace=True)

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 50456 to 46668
Columns: 518 entries, ut_ms to NPWD2882
dtypes: float64(152), int64(366)
memory usage: 267.6 KB


In [91]:
df.head()

,ut_ms,panelsJm2influx,frontJm2influx,backJm2influx,leftJm2influx,rightJm2influx,upJm2influx,downJm2influx,panelssum1Jm2influx_sum,panelssum1decay09Jm2influx_sum,...,NPWD2792,NPWD2801,NPWD2802,NPWD2821,NPWD2851,NPWD2852,NPWD2871,NPWD2872,NPWD2881,NPWD2882
50456,1401004860000,1.933776e+09,1.823922e+09,0.0,7.387109e+05,0.000000e+00,0.000000e+00,7.562989e+08,1.985347e+09,1.786813e+09,...,0.000966,0.001482,0.077523,0.005577,0.364702,0.000506,0.000500,0.000717,0.006478,0.003239
20483,1293102060000,2.436550e+09,2.434897e+09,0.0,2.764297e+07,0.000000e+00,0.000000e+00,8.971499e+07,2.291757e+09,2.062581e+09,...,0.001389,0.001548,0.046193,0.005754,0.072967,0.000537,0.000523,0.000759,0.006778,0.003412
3949,1233579660000,2.402274e+09,2.184793e+09,0.0,1.261050e-09,2.086610e+07,4.723131e+08,1.653141e+08,2.417453e+09,2.175708e+09,...,0.000595,0.001559,0.031102,0.005863,0.312539,0.000539,0.000532,0.000761,0.006748,0.003434
13163,1266750060000,1.786293e+09,1.382542e+09,0.0,3.092967e+07,0.000000e+00,0.000000e+00,9.851794e+08,1.182975e+09,1.064677e+09,...,0.000587,0.001533,0.002286,0.005754,0.134782,0.000531,0.000530,0.000752,0.006780,0.003400
18443,1285758060000,2.119793e+09,2.034129e+09,0.0,4.439690e+06,0.000000e+00,0.000000e+00,5.965231e+08,2.119641e+09,1.907677e+09,...,0.000579,0.001547,0.062628,0.005893,0.262729,0.000532,0.000534,0.000756,0.006789,0.003400


In [94]:
import seaborn as sns


def pics():
    target = df.iloc[:, 485:]

    for x in target:
        plt.figure(figsize=(15, 10))
        sns.scatterplot(data=df, x='ut_ms', y=x)
        plt.show()

#pics()

# Models

In [83]:
X = df.iloc[:, 0:485]
y = df.iloc[:, 485:]

In [84]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2)

# XGBoost

In [96]:
from xgboost import XGBRegressor

lr = MultiOutputRegressor(XGBRegressor())
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

/Users/mymytre/opt/miniconda3/envs/data-science/lib/python3.10/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [97]:
lr.score(X_train, y_train)

0.4393056172238433

In [98]:
y_pred

array([[0.13149847, 0.00152224, 0.17519942, ..., 0.00074506, 0.00675186,
        0.00339637],
       [0.12640545, 0.00152224, 0.17519942, ..., 0.00074506, 0.00675186,
        0.00339637],
       [0.1269817 , 0.00152224, 0.17519942, ..., 0.00074506, 0.00675186,
        0.00339637],
       ...,
       [0.10151224, 0.00152224, 0.17519942, ..., 0.00074506, 0.00675186,
        0.00339637],
       [0.16444927, 0.00152224, 0.17519942, ..., 0.00074506, 0.00675186,
        0.00339637],
       [0.12216077, 0.00152224, 0.17519942, ..., 0.00074506, 0.00675186,
        0.00339637]], dtype=float32)

# GradientBoostingRegressor

In [10]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = MultiOutputRegressor(GradientBoostingRegressor())
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)

In [12]:
gbr.score(X_train, y_train)

0.9846721049963219

In [13]:
y_pred

array([[0.11428143, 0.00153295, 0.17530089, ..., 0.00075073, 0.00677072,
        0.00341156],
       [0.14707627, 0.00153845, 0.17574869, ..., 0.00075396, 0.0066808 ,
        0.00334111],
       [0.163184  , 0.00151858, 0.17582789, ..., 0.00073677, 0.00672855,
        0.00336915],
       ...,
       [0.090444  , 0.00153117, 0.1756967 , ..., 0.00074491, 0.00671712,
        0.00338337],
       [0.13700778, 0.00153906, 0.17576191, ..., 0.00074248, 0.00676117,
        0.00341683],
       [0.11868943, 0.00151042, 0.17543086, ..., 0.00074526, 0.00671098,
        0.00335137]])

# RandomForestRegressor

In [14]:
from sklearn.ensemble import RandomForestRegressor

rfm = MultiOutputRegressor(RandomForestRegressor())
rfm.fit(X_train, y_train)
y_pred = rfm.predict(X_test)

In [15]:
rfm.score(X_train, y_train)

0.8731688311533194

In [111]:
y_pred

array([[0.13149847, 0.00152224, 0.17519942, ..., 0.00074506, 0.00675186,
        0.00339637],
       [0.12640545, 0.00152224, 0.17519942, ..., 0.00074506, 0.00675186,
        0.00339637],
       [0.1269817 , 0.00152224, 0.17519942, ..., 0.00074506, 0.00675186,
        0.00339637],
       ...,
       [0.10151224, 0.00152224, 0.17519942, ..., 0.00074506, 0.00675186,
        0.00339637],
       [0.16444927, 0.00152224, 0.17519942, ..., 0.00074506, 0.00675186,
        0.00339637],
       [0.12216077, 0.00152224, 0.17519942, ..., 0.00074506, 0.00675186,
        0.00339637]], dtype=float32)

# AutoReg model

In [128]:
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.ar_model import AutoReg




model = AutoReg(X_train[1:len(X)], lags=1)
#model_fit = model.fit()
#print('Coefficients: %s' % model_fit.params)
## make predictions
#predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
#for i in range(len(predictions)):
#    print('predicted=%f, expected=%f' % (predictions[i], test[i]))
#rmse = sqrt(mean_squared_error(test, predictions))
#print('Test RMSE: %.3f' % rmse)
## plot results
#pyplot.plot(test)
#pyplot.plot(predictions, color='red')
#pyplot.show()

/Users/mymytre/opt/miniconda3/envs/data-science/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


TypeError: can only concatenate list (not "str") to list